In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#JDBC CONFIG
jdbc_url = (
    "jdbc:sqlserver://sqlsrv-fr-risk-neu.database.windows.net:1433;"
    "database=sqldb-fr-risk;"
    "encrypt=true;"
    "trustServerCertificate=false;"
    "hostNameInCertificate=*.database.windows.net;"
    "loginTimeout=30;"
)

props = {
    "user": "CloudSA15903b81",
    "password": "Fraud@123",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
}

spark._jvm.java.lang.Class.forName(props["driver"])

JavaObject id=o447

In [0]:
#HELPERS
def null_if_empty(c):
    return F.when(F.trim(F.col(c)).isin("", "null", "NULL"), None).otherwise(F.col(c))

def to_bool(c):
    t = ["1","true","t","yes","y","Y","True","TRUE"]
    f = ["0","false","f","no","n","N","False","FALSE"]
    return (
        F.when(F.trim(F.col(c)).isin(*t), F.lit(True))
         .when(F.trim(F.col(c)).isin(*f), F.lit(False))
         .otherwise(None)
    )

def ts(colname):
    """
    Try multiple datetime patterns (including ones with timezone),
    then fall back to Spark's generic to_timestamp.
    """
    col = F.col(colname)

    patterns = [
        "yyyy-MM-dd HH:mm:ss.SSS",
        "yyyy-MM-dd HH:mm:ss",
        "yyyy-MM-dd'T'HH:mm:ss.SSS",
        "yyyy-MM-dd'T'HH:mm:ss",
        "yyyy-MM-dd'T'HH:mm:ssXXX",   # ISO8601 with timezone, e.g. 2025-11-03T10:23:11+00:00
        "MM/dd/yyyy HH:mm:ss",
        "yyyy-MM-dd",
    ]

    expr = F.lit(None).cast("timestamp")
    for p in patterns:
        expr = F.coalesce(expr, F.to_timestamp(col, p))

    # final fallback: let Spark try to parse without an explicit pattern
    expr = F.coalesce(expr, F.to_timestamp(col))

    return expr


def write_silver(df, table_name):
    """
    Writes a cleaned DF into silver.<table_name> in SQL.
    For now: overwrite each run (full reload from STG).
    """
    target = f"silver.{table_name}"
    (df.write
        .format("jdbc")
        .option("url", jdbc_url)
        .options(**props)
        .option("dbtable", target)
        .mode("overwrite")      # simple full refresh for now
        .save())
    print(f"Written {df.count()} rows to {target}")

from pyspark.sql import functions as F

def dq_summary(name: str, df_stg, df_silver, key_col: str | None = None):
    """
    Simple data quality summary:
    - Row counts in STG vs SILVER
    - Optional distinct key counts
    """
    stg_count = df_stg.count()
    silver_count = df_silver.count()

    print(f"\n================ DQ SUMMARY: {name} ================")
    print(f"STG rows    : {stg_count}")
    print(f"SILVER rows : {silver_count}")

    if key_col and key_col in df_stg.columns and key_col in df_silver.columns:
        stg_keys = df_stg.select(key_col).distinct().count()
        silver_keys = df_silver.select(key_col).distinct().count()
        print(f"STG distinct {key_col}    : {stg_keys}")
        print(f"SILVER distinct {key_col} : {silver_keys}")

    if stg_count == silver_count:
        print("ROW CHECK   : OK (same row count)")
    elif silver_count < stg_count:
        print("ROW CHECK   : WARNING – SILVER has fewer rows than STG")
    else:
        print("ROW CHECK   : NOTE – SILVER has more rows than STG (e.g. exploded records)")


def dq_null_ratios(name: str, df_silver, cols: list[str], sample_rows: int = 1000000):
    """
    Prints null ratios for selected columns in the SILVER table.
    To avoid huge scans, you can limit rows via sample_rows if table is massive.
    """
    print(f"\n---------------- NULL RATIO CHECK: {name} ----------------")
    total = df_silver.limit(sample_rows).count()
    if total == 0:
        print("No rows in SILVER table.")
        return

    for c in cols:
        if c not in df_silver.columns:
            print(f"{c}: (column not found)")
            continue
        nulls = df_silver.filter(F.col(c).isNull()).count()
        pct = (nulls / total) * 100
        print(f"{c}: {nulls}/{total} nulls ({pct:.2f}%)")


In [0]:
#1) TRANSACTIONS
df_trx_stg = spark.read.jdbc(jdbc_url, "stg.transactions", properties=props)

cols_trx_trim = [
    "txn_id","account_id","card_id","merchant_id","device_id",
    "txn_dt","amount","currency","country_iso","city","channel","entry_mode","ip",
    "distance_km_prev","txn_in_last_min5",
    "is_international","is_new_device","is_night","is_high_amount","is_fraud"
]

for c in cols_trx_trim:
    if c in df_trx_stg.columns:
        df_trx_stg = df_trx_stg.withColumn(c, null_if_empty(c))

df_trx = (
    df_trx_stg
      # keep a copy of the raw string for debugging / lineage
      .withColumn("txn_dt_raw", F.col("txn_dt"))
      # parse into proper timestamp
      .withColumn("txn_dt", ts("txn_dt"))
      .withColumn("amount", F.round(F.col("amount").cast("double"), 2))
      .withColumn("distance_km_prev", F.col("distance_km_prev").cast("double"))
      .withColumn("txn_in_last_min5", F.col("txn_in_last_min5").cast("int"))
      .withColumn("is_international", to_bool("is_international"))
      .withColumn("is_new_device",    to_bool("is_new_device"))
      .withColumn("is_night",         to_bool("is_night"))
      .withColumn("is_high_amount",   to_bool("is_high_amount"))
      .withColumn("is_fraud",         to_bool("is_fraud"))
)

w_trx = Window.partitionBy("txn_id").orderBy(F.col("txn_dt").desc_nulls_last())

df_trx_final = (
    df_trx.filter(F.col("txn_id").isNotNull())
          .withColumn("_rn", F.row_number().over(w_trx))
          .filter(F.col("_rn") == 1)
          .drop("_rn")
          .withColumn("loaded_at", F.current_timestamp())
)

write_silver(df_trx_final, "transactions")

# Re-read silver.transactions for DQ checks
df_trx_silver = spark.read.jdbc(jdbc_url, "silver.transactions", properties=props)

dq_summary(
    name="transactions",
    df_stg=df_trx_stg,
    df_silver=df_trx_silver,
    key_col="txn_id"
)

dq_null_ratios(
    name="transactions",
    df_silver=df_trx_silver,
    cols=[
        "txn_dt",
        "amount",
        "account_id",
        "merchant_id",
        "device_id",
        "is_fraud"
    ]
)

Written 594609 rows to silver.transactions

================ DQ SUMMARY: transactions ================
STG rows    : 1482774
SILVER rows : 594609
STG distinct txn_id    : 594609
SILVER distinct txn_id : 594609
ROW CHECK   : WARNING – SILVER has fewer rows than STG

---------------- NULL RATIO CHECK: transactions ----------------
txn_dt: 17994/594609 nulls (3.03%)
amount: 17828/594609 nulls (3.00%)
account_id: 0/594609 nulls (0.00%)
merchant_id: 0/594609 nulls (0.00%)
device_id: 0/594609 nulls (0.00%)
is_fraud: 0/594609 nulls (0.00%)


In [0]:
#2) ACCOUNTS
df_acc_stg = spark.read.jdbc(jdbc_url, "stg.accounts", properties=props)

# Basic cleaning – you can refine later as we know exact schema
df_acc = df_acc_stg
for c in df_acc.columns:
    df_acc = df_acc.withColumn(c, null_if_empty(c))

# Example: cast maybe numeric-ish cols if you have them (placeholder)
# df_acc = df_acc.withColumn("age", F.col("age").cast("int"))

df_acc = df_acc.withColumn("loaded_at", F.current_timestamp())
write_silver(df_acc, "accounts")

df_acc_silver = spark.read.jdbc(jdbc_url, "silver.accounts", properties=props)

dq_summary(
    name="accounts",
    df_stg=df_acc_stg,
    df_silver=df_acc_silver,
    key_col="account_id"   # adjust if your key has different name
)

dq_null_ratios(
    name="accounts",
    df_silver=df_acc_silver,
    cols=[
        "account_id",
        "customer_name",
        "email",
        "country_iso",
        "created_dt",
        "status",
        "id",
        "loaded_at"  # whatever important columns you have
    ]
)


Written 700000 rows to silver.accounts

================ DQ SUMMARY: accounts ================
STG rows    : 700000
SILVER rows : 700000
STG distinct account_id    : 100000
SILVER distinct account_id : 100000
ROW CHECK   : OK (same row count)

---------------- NULL RATIO CHECK: accounts ----------------
account_id: 0/700000 nulls (0.00%)
customer_name: 21228/700000 nulls (3.03%)
email: 21116/700000 nulls (3.02%)
country_iso: 20476/700000 nulls (2.93%)
created_dt: 21164/700000 nulls (3.02%)
status: 21148/700000 nulls (3.02%)
id: 0/700000 nulls (0.00%)
loaded_at: 0/700000 nulls (0.00%)


In [0]:
#3) MERCHANTS
df_mer_stg = spark.read.jdbc(jdbc_url, "stg.merchants", properties=props)

for c in df_mer_stg.columns:
    df_mer_stg = df_mer_stg.withColumn(c, null_if_empty(c))

df_mer = df_mer_stg.withColumn("loaded_at", F.current_timestamp())
write_silver(df_mer, "merchants")

df_merch_silver = spark.read.jdbc(jdbc_url, "silver.merchants", properties=props)

dq_summary(
    name="merchants",
    df_stg=df_mer_stg,
    df_silver=df_merch_silver,
    key_col="merchant_id"
)

dq_null_ratios(
    name="merchants",
    df_silver=df_merch_silver,
    cols=[
        "merchant_id",
        "category_mcc",
        "risk_bucket",
        "country_iso",
        "avg_ticket",
        "velocity_cap"
    ]
)


Written 110000 rows to silver.merchants

================ DQ SUMMARY: merchants ================
STG rows    : 110000
SILVER rows : 110000
STG distinct merchant_id    : 20000
SILVER distinct merchant_id : 20000
ROW CHECK   : OK (same row count)

---------------- NULL RATIO CHECK: merchants ----------------
merchant_id: 0/110000 nulls (0.00%)
category_mcc: 3183/110000 nulls (2.89%)
risk_bucket: 3358/110000 nulls (3.05%)
country_iso: 3347/110000 nulls (3.04%)
avg_ticket: 3069/110000 nulls (2.79%)
velocity_cap: 3335/110000 nulls (3.03%)


In [0]:
#4) DEVICES
df_dev_stg = spark.read.jdbc(jdbc_url, "stg.devices", properties=props)

for c in df_dev_stg.columns:
    df_dev_stg = df_dev_stg.withColumn(c, null_if_empty(c))

df_dev = df_dev_stg.withColumn("loaded_at", F.current_timestamp())
write_silver(df_dev, "devices")
df_dev_silver = spark.read.jdbc(jdbc_url, "silver.devices", properties=props)

dq_summary(
    name="devices",
    df_stg=df_dev_stg,
    df_silver=df_dev_silver,
    key_col="device_id"
)

dq_null_ratios(
    name="devices",
    df_silver=df_dev_silver,
    cols=[
        "device_id",
        "device_type",
        "os",
        "first_seen_dt",
        "is_rooted"
    ]
)

Written 840000 rows to silver.devices

================ DQ SUMMARY: devices ================
STG rows    : 840000
SILVER rows : 840000
STG distinct device_id    : 120000
SILVER distinct device_id : 120000
ROW CHECK   : OK (same row count)

---------------- NULL RATIO CHECK: devices ----------------
device_id: 0/840000 nulls (0.00%)
device_type: 25378/840000 nulls (3.02%)
os: 25969/840000 nulls (3.09%)
first_seen_dt: 24859/840000 nulls (2.96%)
is_rooted: 25129/840000 nulls (2.99%)


In [0]:
#5) LOGIN EVENTS
df_login_stg = spark.read.jdbc(jdbc_url, "stg.login_events", properties=props)

for c in df_login_stg.columns:
    df_login_stg = df_login_stg.withColumn(c, null_if_empty(c))

# event_dt is already datetime2 in stg because we altered table,
# but if it's string in some rows, we can safely try to parse:
if "event_dt" in df_login_stg.columns:
    df_login_stg = df_login_stg.withColumn("event_dt", ts("event_dt"))

df_login = df_login_stg.withColumn("loaded_at", F.current_timestamp())
write_silver(df_login, "login_events")

df_login_silver = spark.read.jdbc(jdbc_url, "silver.login_events", properties=props)

dq_summary(
    name="login_events",
    df_stg=df_login_stg,
    df_silver=df_login_silver,
    key_col="event_id"   # or the proper key column
)

dq_null_ratios(
    name="login_events",
    df_silver=df_login_silver,
    cols=[
        "event_id",
        "customer_id",
        "device_id",
        "event_dt",
        "success",
        "ip",        # optional but useful
        "mfa_used"   # optional but useful
    ]
)


Written 17562 rows to silver.login_events

================ DQ SUMMARY: login_events ================
STG rows    : 17562
SILVER rows : 17562
STG distinct event_id    : 17562
SILVER distinct event_id : 17562
ROW CHECK   : OK (same row count)

---------------- NULL RATIO CHECK: login_events ----------------
event_id: 0/17562 nulls (0.00%)
customer_id: 0/17562 nulls (0.00%)
device_id: 0/17562 nulls (0.00%)
event_dt: 564/17562 nulls (3.21%)
success: 521/17562 nulls (2.97%)
ip: 0/17562 nulls (0.00%)
mfa_used: 532/17562 nulls (3.03%)


In [0]:
#6) CHARGEBACKS
df_cb_stg = spark.read.jdbc(jdbc_url, "stg.chargebacks", properties=props)

for c in df_cb_stg.columns:
    df_cb_stg = df_cb_stg.withColumn(c, null_if_empty(c))

if "filed_dt" in df_cb_stg.columns:
    df_cb_stg = df_cb_stg.withColumn("filed_dt", ts("filed_dt"))

df_cb = df_cb_stg.withColumn("loaded_at", F.current_timestamp())
write_silver(df_cb, "chargebacks")

df_cb_silver = spark.read.jdbc(jdbc_url, "silver.chargebacks", properties=props)

dq_summary(
    name="chargebacks",
    df_stg=df_cb_stg,
    df_silver=df_cb_silver,
    key_col="cb_id"   # or use txn_id if you want that as main key
)

dq_null_ratios(
    name="chargebacks",
    df_silver=df_cb_silver,
    cols=[
        "cb_id",
        "txn_id",
        "reason_code",
        "filed_dt",
        "won_dispute"
    ]
)


Written 264012 rows to silver.chargebacks

================ DQ SUMMARY: chargebacks ================
STG rows    : 264012
SILVER rows : 264012
STG distinct cb_id    : 44002
SILVER distinct cb_id : 44002
ROW CHECK   : OK (same row count)

---------------- NULL RATIO CHECK: chargebacks ----------------
cb_id: 0/264012 nulls (0.00%)
txn_id: 0/264012 nulls (0.00%)
reason_code: 8049/264012 nulls (3.05%)
filed_dt: 7507/264012 nulls (2.84%)
won_dispute: 7788/264012 nulls (2.95%)


In [0]:
#7) IP REPUTATION
df_ip_stg = spark.read.jdbc(jdbc_url, "stg.ip_reputation", properties=props)

for c in df_ip_stg.columns:
    df_ip_stg = df_ip_stg.withColumn(c, null_if_empty(c))

df_ip = df_ip_stg.withColumn("loaded_at", F.current_timestamp())
write_silver(df_ip, "ip_reputation")

df_ip_silver = spark.read.jdbc(jdbc_url, "silver.ip_reputation", properties=props)

dq_summary(
    name="ip_reputation",
    df_stg=df_ip_stg,
    df_silver=df_ip_silver,
    key_col="ip"
)

dq_null_ratios(
    name="ip_reputation",
    df_silver=df_ip_silver,
    cols=[
        "ip",
        "asn",
        "proxy_type",
        "reputation_score",
        "updated_dt"
    ]
)

Written 100000 rows to silver.ip_reputation

================ DQ SUMMARY: ip_reputation ================
STG rows    : 100000
SILVER rows : 100000
STG distinct ip    : 100000
SILVER distinct ip : 100000
ROW CHECK   : OK (same row count)

---------------- NULL RATIO CHECK: ip_reputation ----------------
ip: 0/100000 nulls (0.00%)
asn: 2990/100000 nulls (2.99%)
proxy_type: 3010/100000 nulls (3.01%)
reputation_score: 2962/100000 nulls (2.96%)
updated_dt: 3040/100000 nulls (3.04%)
